In [8]:
%run load_modules.ipynb

Loading BokehJS ...

In [10]:
kB = 1.381e-23 # Boltzmann Constant
T = 300 # Temperature
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 10 # Post Load Amplifier Gain
bandwidth0 = 1250 # Unfiltered Bandwdith in GHz
Pave = 10e-6 # Average signal power in Watt
Nsamp=1000

sigmaT = np.sqrt(4*kB*T/RL*bandwidth0*1e9)*G
sigma0 = sigmaT; sigma1 = sigmaT
v1=np.zeros(Nsamp); v2=np.zeros(Nsamp); v3=np.zeros(Nsamp)
v = np.zeros(3*Nsamp)

bandwidth = 20.0 # Filtered receiver bandwidth in GHz

t=np.linspace(0,1200,3*Nsamp) # time in picoseconds
Threshold=np.zeros(3*Nsamp)+Pave*R*RL*G

p = figure(width=400, height=400, background_fill_color="lightgray", title='Signal with Amplitude Noise')
lt=p.line(t,Threshold*1e6,line_dash='dashed',line_color='red')
p.title.text_font_style='normal'; p.title.text_font_size='12pt'
p.outline_line_width = 1; p.outline_line_color = "black"; p.min_border_top = 0
l=p.line(t,v,line_color='#2222aa')
lt=p.line(t,Threshold*1e6,line_dash='dashed',line_color='red')
p.x_range = Range1d(0, 10)
p.xaxis.axis_label="Time (psec)"; p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_style = "normal"; p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label="Voltage (mV)"; p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_style = "normal"; p.yaxis.axis_label_text_font_size = "12pt";

p.add_layout(Label(x=200, y=300, text='0', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))
p.add_layout(Label(x=600, y=300, text='1', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))
p.add_layout(Label(x=1000, y=300, text='0', text_color='red', text_align='center', text_font_size='14pt',
                   background_fill_color='lightgray', x_units='data', y_units='screen'))

Vvalue=np.linspace(-4*1e-3,21*1e-3,100)
p1 = figure(width=400, height=400, background_fill_color="lightgray", title='Noise Distribution', y_axis_type='linear')
p1.title.text_font_style='normal'; p1.title.text_font_size='12pt'
p1.outline_line_width = 1; p1.outline_line_color = "black"
p1.y_range = Range1d(1e-6,350)
vline = Span(location=0.2,dimension='height', line_color='red', line_dash='dashed')
p1.add_layout(vline)
hist1, edges1 = np.histogram(v1, density=True, bins=100)
hist2, edges2 = np.histogram(v2, density=True, bins=100)
q1 = p1.quad(top=hist1, bottom=0.001, left=edges1[:-1], right=edges1[1:],
        fill_color="#036564", line_color="#033649")
q2 = p1.quad(top=hist2, bottom=0.001, left=edges2[:-1], right=edges2[1:],
        fill_color="#036564", line_color="#033649")
ql1 = p1.line(Vvalue, (1/(sigma0*1e5*(2*np.pi)**0.5)*np.exp(-(Vvalue*1e-3-0)**2/(sigma0)**2)), line_width=2, line_color='darkorange')
ql2 = p1.line(Vvalue, (1/(sigma1*1e5*(2*np.pi)**0.5)*np.exp(-(Vvalue*1e-3-Pave*2*R*RL*G)**2/(sigma1)**2)), line_width=2, line_color='darkorange')
p1.xaxis.axis_label="Voltage (V)"; p1.xaxis.major_label_text_font_size = "12pt"
p1.xaxis.axis_label_text_font_style = "normal"; p1.xaxis.axis_label_text_font_size = "12pt"
p1.yaxis.axis_label="Probabilty Density"; p1.yaxis.major_label_text_font_size = "12pt"
p1.yaxis.axis_label_text_font_style = "normal"; p1.yaxis.axis_label_text_font_size = "12pt"

label=Label(x=700, y=20, text='Q=', text_color='darkblue', background_fill_color='lightgray')
label1=Label(x=40, y=Pave*R*RL*G*1e6, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
label2=Label(x=700,  y=0, text='BER=',
                            text_color='darkblue', background_fill_color='lightgray', y_units='screen')
label3=Label(x=Pave*R*RL*G, y=1100, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
p.add_layout(label2); p1.add_layout(label3)
p1.add_layout(label); p.add_layout(label1)

Pave_slider=FloatSlider(min=0, max=35, step=0.1, value = 10, description='Pave (\u03BCW)', continuous_update=True)

def replot(Pave):
    Pave = Pave*1e-3 # Give power in milliwatts
    vhist1=np.zeros(Nsamp)
    vhist2=np.zeros(Nsamp)
    for i in range (Nsamp):
        v1 = np.random.normal(0, sigma0*RL, Nsamp)
        v2 = np.random.normal(Pave*2*R*RL*G, sigma0*RL, Nsamp)
        v3 = np.random.normal(0, sigma0*RL, Nsamp)
        v = np.concatenate([v1,v2,v3])
        sig_fft= np.fft.rfft(v)
        sig_fft_norm = np.absolute(sig_fft)/np.amax(np.absolute(sig_fft))
        fft_freq = np.linspace(0,3*Nsamp/2/1.200,int(3*Nsamp/2+1))
        filter=np.heaviside(bandwidth-fft_freq, 0.5)
        sig_fft_f = sig_fft*filter
        v=(np.fft.irfft(sig_fft_f))
        vhist1[i]=v[int(Nsamp/2)]
        vhist2[i]=v[int(3*Nsamp/2)]

    hist1, edges1 = np.histogram(vhist1, density=True, bins=100)
    hist2, edges2 = np.histogram(vhist2, density=True, bins=100)
    
    sigmaTR = np.sqrt(4*kB*T/RL*bandwidth*1e9)*G
    sigma0R = sigmaTR; sigma1R = sigmaTR

    lt.data_source.data['y']=(np.zeros(3*Nsamp)+Pave*R*RL*G).tolist()
    l.data_source.data['y']=v.tolist()
    q1.data_source.data['top']=hist1/np.max(hist1)*1/((sigma0R*RL)*(2*np.pi)**0.5)
    q2.data_source.data['top']=hist2/np.max(hist2)*1/((sigma1R*RL)*(2*np.pi)**0.5)
    q1.data_source.data['left']=edges1[:-1]
    q2.data_source.data['left']=edges2[:-1]
    q1.data_source.data['right']=right=edges1[1:]
    q2.data_source.data['right']=right=edges2[1:]
    ql1.data_source.data['y']=1/((sigma0R*RL)*(2*np.pi)**0.5)*np.exp(-(Vvalue-0)**2/2/(sigma0R*RL)**2)
    ql2.data_source.data['y']=1/((sigma1R*RL)*(2*np.pi)**0.5)*np.exp(-(Vvalue-Pave*2*R*RL*G)**2/2/(sigma1R*RL)**2)
    vline.location=Pave*R*RL*G
    Q = (2*Pave*R*G)/(sigma0R+sigma1R); label.text='Q-Factor='+ str(Q)[:3]
    label1.y=Pave*R*RL*G
    BER = 0.5*special.erfc(Q/np.sqrt(2))
    label2.text='BER='+'{:.2e}'.format(BER)
    label3.x=Pave*R*RL*G
    push_notebook(handle=fig_handle)

fig_handle = show(row(p,p1), notebook_handle=True)

interact(replot, Pave=Pave_slider);

interactive(children=(FloatSlider(value=10.0, description='Pave (μW)', max=35.0), Output()), _dom_classes=('wi…